In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
from pylab import plt
import scipy.optimize as sco
plt.style.use("ggplot")

In [2]:
risk_free_rate = 0.03

stocks = ["HSBA.L", "RR.L", "BP.L", "BLND.L", "AV.L"]
num_assets = len(stocks)

yf_stock_data = yf.download(" ".join(stocks), start="2024-01-01", end="2024-06-01", rounding=True)["Close"]
yf_stock_data.to_csv("yf_stock_data.csv", index=True, header=True)

ftse_index = yf.download("^FTSE", start="2024-01-01", end="2024-06-01", progress=False, rounding=True)
ftse_index.to_csv("ftse_index.csv", index=True, header=True)

[*********************100%%**********************]  5 of 5 completed


In [3]:
raw_stock_data = pd.DataFrame(yf_stock_data)
raw_stock_data.fillna(method="ffill", inplace=True)
clean_stock_data = raw_stock_data

raw_ftse_data = pd.DataFrame(ftse_index)
raw_ftse_data.fillna(method="ffill", inplace=True)
clean_ftse_data = raw_ftse_data

clean_stock_data.head(), clean_ftse_data.head()

/var/folders/52/118zx6s92x5dv1www3l374100000gn/T/ipykernel_17095/2942537575.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  raw_stock_data.fillna(method="ffill", inplace=True)
/var/folders/52/118zx6s92x5dv1www3l374100000gn/T/ipykernel_17095/2942537575.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  raw_ftse_data.fillna(method="ffill", inplace=True)


(             AV.L  BLND.L    BP.L  HSBA.L   RR.L
 Date                                            
 2024-01-02  433.7   397.3  471.00   630.5  298.1
 2024-01-03  429.8   394.9  471.95   625.4  294.8
 2024-01-04  433.0   395.8  473.45   632.5  297.8
 2024-01-05  433.3   395.4  473.85   634.3  296.6
 2024-01-08  435.9   400.5  461.45   640.0  305.2,
               Open    High     Low   Close  Adj Close      Volume
 Date                                                             
 2024-01-02  7733.2  7764.4  7689.0  7721.5     7721.5   678041300
 2024-01-03  7721.5  7744.6  7650.3  7682.3     7682.3   857615300
 2024-01-04  7682.3  7728.2  7678.8  7723.1     7723.1   963235800
 2024-01-05  7723.1  7723.1  7642.9  7689.6     7689.6   701524000
 2024-01-08  7689.6  7694.2  7636.1  7694.2     7694.2  1055586300)